In [39]:
import os
import uuid
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from openai import OpenAI

# === Config ===
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "multi_user_docs"
EMBEDDING_DIM = 1536
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

client = QdrantClient(url=QDRANT_URL)
openai_client = OpenAI(api_key=OPENAI_KEY)

def ingest_user_pdfs(user_id: str, pdf_dir: str):
    if not client.collection_exists(COLLECTION_NAME):
        client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=EMBEDDING_DIM, distance=Distance.COSINE)
        )

    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=250)

    for file in os.listdir(pdf_dir):
        if not file.endswith(".pdf"):
            continue

        full_path = os.path.join(pdf_dir, file)
        reader = PdfReader(full_path)
        raw_text = "\n".join([page.extract_text() or "" for page in reader.pages])
        chunks = splitter.split_text(raw_text)

        # Embed with OpenAI
        response = openai_client.embeddings.create(model="text-embedding-3-small", input=chunks)
        vectors = [r.embedding for r in response.data]

        points = [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=vec,
                payload={
                    "content": chunk,
                    "user_id": user_id,
                    "doc_id": file,
                    "name": file,
                    "usage":"document",
                    "meta_data": {"source": file}  
                }
            )
            for chunk, vec in zip(chunks, vectors)
        ]

        client.upsert(collection_name=COLLECTION_NAME, points=points)
        print(f"✅ Indexed {len(points)} chunks from {file}")


In [ ]:
from agno.knowledge import AgentKnowledge
from agno.embedder.openai import OpenAIEmbedder
from agno.vectordb.qdrant import Qdrant as AgnoQdrant
from agno.models.openai import OpenAIChat
from agno.agent import Agent

def create_user_agent(user_id: str):
    # Create vector DB (Agno wrapper)
    vector_db = AgnoQdrant(
        collection=COLLECTION_NAME,
        url=QDRANT_URL
    )

    # Set up knowledge with user_id filter
    knowledge = AgentKnowledge(
        vector_db=vector_db,
        embedder=OpenAIEmbedder(),
        user_id=user_id  # ⬅️ this enforces secure filtering
    )

    # Create agent
    agent = Agent(
        model=OpenAIChat(id="gpt-3.5-turbo-0125"),
        knowledge=knowledge,
        search_knowledge=True,
        user_id=user_id,
        instructions=[
            "You are a helpful assistant that can answer questions based on the provided PDF documents.",
            "Always cite the source when providing information from the documents.",
            "If you cannot find the answer in the documents, clearly state that.",
            "Be concise but thorough in your responses.",
            "Cite sources when possible. If nothing is found, say so clearly."
        ]
    )
    return agent


In [44]:
def run_chat():
    user_id = input("Enter your user ID: ").strip()
    user_folder = f"data/{user_id}"

    if not os.path.isdir(user_folder):
        print("❌ No PDF directory found for this user.")
        return

    # Ingest files (only re-run if new docs added)
    ingest_user_pdfs(user_id, user_folder)

    # Create secure agent
    agent = create_user_agent(user_id)

    print(f"\n🤖 Agent ready for user {user_id}. Ask questions (type 'exit' to quit).")
    while True:
        q = input(f"[{user_id}]> ")
        if q.strip().lower() in {"exit", "quit"}:
            break
        response = agent.run(q)
        print("🧠", response.content)

run_chat()


✅ Indexed 45 chunks from Introduction.pdf


INFO Embedder not provided, using OpenAIEmbedder as default.


🤖 Agent ready for user user_123. Ask questions (type 'exit' to quit).
🧠 The plan for Week 5 includes the following tasks:
- Set Up Supabase Project
  - Create tables: Users, Projects, Milestones, Investments (if off-chain logging is needed).
  - Configure Supabase Auth for role-based access (developer, investor, council).
- Direct Integration
  - Frontend calls to Supabase for reading/writing project data, user profiles.
  - Real-time updates if desired (e.g., project status changes).

(Source: Plan of Action.pdf)
🧠 Akreage refers to a decentralized crowdfunding platform designed to bridge the gap between real estate developers and global investors by leveraging blockchain technology. It introduces two digital assets: Akreage Dollar (AUSD) and Akres Token. AUSD is a USD-backed stablecoin that allows developers to raise funds in a secure and predictable manner, while Akres Token is the governance token that powers the Akreage ecosystem, enabling democratic decision-making and decentral